# Remapping of WOA 2013 T/S for bias plots

In [1]:
import xarray as xr
from vcr import utils, conserve
import numpy as np
import seawater
import time
import pydap

url for the WOA13 dataset:

In [2]:
url_temp = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/archive/data/0114815/public/temperature/netcdf/decav/1.00/'
url_salt = 'https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/archive/data/0114815/public/salinity/netcdf/decav/1.00/'

The target vertical interfaces can be loaded from MOM6 z-level outputs. 

In order not to be dependent on model outputs in the notebook,
we just copy the values as found in model files:

In [3]:
depth_tgt = np.array([2.5000e+00, 1.0000e+01, 2.0000e+01, 3.2500e+01, 5.1250e+01,
                      7.5000e+01, 1.0000e+02, 1.2500e+02, 1.5625e+02, 2.0000e+02,
                      2.5000e+02, 3.1250e+02, 4.0000e+02, 5.0000e+02, 6.0000e+02,
                      7.0000e+02, 8.0000e+02, 9.0000e+02, 1.0000e+03, 1.1000e+03,
                      1.2000e+03, 1.3000e+03, 1.4000e+03, 1.5375e+03, 1.7500e+03,
                      2.0625e+03, 2.5000e+03, 3.0000e+03, 3.5000e+03, 4.0000e+03,
                      4.5000e+03, 5.0000e+03, 5.5000e+03, 6.0000e+03, 6.5000e+03])


depth_bnds_tgt = np.array([0.000e+00, 5.000e+00, 1.500e+01, 2.500e+01, 4.000e+01, 6.250e+01,
                           8.750e+01, 1.125e+02, 1.375e+02, 1.750e+02, 2.250e+02, 2.750e+02,
                           3.500e+02, 4.500e+02, 5.500e+02, 6.500e+02, 7.500e+02, 8.500e+02,
                           9.500e+02, 1.050e+03, 1.150e+03, 1.250e+03, 1.350e+03, 1.450e+03,
                           1.625e+03, 1.875e+03, 2.250e+03, 2.750e+03, 3.250e+03, 3.750e+03,
                           4.250e+03, 4.750e+03, 5.250e+03, 5.750e+03, 6.250e+03, 6.750e+03])

In [4]:
def process_woa_TS_data(depth_tgt, depth_bnds_tgt, period=0):
    """  download data from opendap, compute potential temperature,
    roll to 0-360 and create dataset """
    
    cperiod = str(period).zfill(2)
    # load the original data
    woa13_t = xr.open_dataset(f'{url_temp}/woa13_decav_t{cperiod}_01.nc',
                              decode_times=False, engine='pydap')
    woa13_s = xr.open_dataset(f'{url_salt}/woa13_decav_s{cperiod}_01.nc',
                              decode_times=False, engine='pydap')
    
    # compute potential temperature
    p = xr.apply_ufunc(seawater.eos80.pres, woa13_t.depth, woa13_t.t_an,
                       dask='parallelized',
                       output_dtypes=[woa13_t.t_an.dtype])
    ptemp = xr.apply_ufunc(seawater.eos80.ptmp, woa13_s.s_an, woa13_t.t_an, p,
                           dask='parallelized',
                           output_dtypes=[woa13_t.t_an.dtype])
    
    # re-arange depth bounds for WOA13
    depth_bnds_src = utils.bounds_2d_to_1d(woa13_t['depth_bnds'])
    
    # create remapping weights
    remapping = conserve.create_remapping_matrix(depth_bnds_src, depth_bnds_tgt,
                                                 strict=False)
    
    # Remap the data
    ptemp_remapped = conserve.vertical_remap_z2z(ptemp.squeeze(dim='time').values,
                                                 remapping)
    salt_remapped = conserve.vertical_remap_z2z(woa13_s['s_an'].squeeze(dim='time').values,
                                                remapping)
    
    # roll the data
    lon = np.roll(woa13_t['lon'].values, -180, axis=0)
    lon = np.mod(lon+360, 360)
    ptemp_0360 = np.roll(ptemp_remapped, -180, axis=-1)
    salt_0360 = np.roll(salt_remapped, -180, axis=-1)
    
    ptemp_0360 = np.expand_dims(ptemp_0360, axis=0)
    salt_0360 = np.expand_dims(salt_0360, axis=0)
    
    woa13_remapped = xr.Dataset()
    woa13_remapped['z_l'] = xr.DataArray(data=depth_tgt, dims=('z_l'))
    woa13_remapped['lon'] = xr.DataArray(data=lon, dims=('lon'))
    woa13_remapped['lat'] = xr.DataArray(data=woa13_t['lat'].values, dims=('lat'))
    woa13_remapped['time'] = woa13_t['time']
    woa13_remapped['ptemp'] = xr.DataArray(data=ptemp_0360,
                                           coords={'time': woa13_remapped['time'],
                                                   'z_l': woa13_remapped['z_l'],
                                                   'lat': woa13_remapped['lat'],
                                                   'lon': woa13_remapped['lon']},
                                           dims=('time', 'z_l', 'lat', 'lon'),
                                           attrs={'_FillValue': 1.0e+20})
    woa13_remapped['salinity'] = xr.DataArray(data=salt_0360,
                                              coords={'time': woa13_remapped['time'],
                                                      'z_l': woa13_remapped['z_l'],
                                                      'lat': woa13_remapped['lat'],
                                                      'lon': woa13_remapped['lon']},
                                              dims=('time', 'z_l', 'lat', 'lon'),
                                              attrs={'_FillValue': 1.0e+20})
    
    woa13_remapped['ptemp'] = woa13_remapped['ptemp'].fillna(1.0e+20)
    woa13_remapped['salinity'] = woa13_remapped['salinity'].fillna(1.0e+20)
    
    
    return woa13_remapped

## Annual data

In [5]:
period = 0
ds = process_woa_TS_data(depth_tgt, depth_bnds_tgt)
ds.to_netcdf('WOA13_ptemp+salinity_annual_35levels.nc',
             format='NETCDF3_64BIT')
ds.close()

time.sleep(2)

## Monthly data

In [6]:
for kt in range(12):
    month = kt + 1
    cmonth = str(month).zfill(2)
    ds = process_woa_TS_data(depth_tgt, depth_bnds_tgt, period=month)
    ds.to_netcdf(f'WOA13_ptemp+salinity_m{cmonth}_35levels.nc',
                 format='NETCDF3_64BIT')
    ds.close()
    time.sleep(2)

In [7]:
dsm = xr.open_mfdataset('WOA13_ptemp+salinity_m*_35levels.nc',
                        combine='by_coords', decode_times=False)
dsm.to_netcdf('WOA13_ptemp+salinity_monthly_35levels.nc',
              format='NETCDF3_64BIT')

*time at approx 30 seconds run time*